<a href="https://colab.research.google.com/github/CAVASOL/aiffel_quest/blob/main/Exploration_quest/exploration_6/xp6_deploy_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 14-1. 프로젝트: CIFAR-10 모델 튜닝하고 배포하기!

**Goal**

`The goal involves choosing a project, adjusting its parameters precisely, and deploying it using TFServing and Docker.` The focus is on optimizing model performance by finetuning parameters and ensuring seamless deployment for efficient functionality within a Docker container.

**Index**

    1. Finetune hyperparameter
    2. Deploy the model
    3. Create a TFLite model
    4. Conclusion

### 1. Finetune hyperparameter

In [ ]:
pip install keras-tuner

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import keras
import keras_tuner as kt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.datasets import cifar10
import matplotlib.pyplot as plt

In [ ]:
print("Keras", keras.__version__)

Keras 2.6.0


In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [ ]:
X_train = x_train.reshape(-1, 32, 32, 3)
X_test = x_test.reshape(-1, 32, 32, 3)
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

In [ ]:
print(X_train.shape)

(50000, 32, 32, 3)


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2)

In [ ]:
print(X_train.shape)

(40000, 32, 32, 3)


In [ ]:
class DeepTuner(kt.Tuner):
    def run_trial(self, trial, X, y, validation_data, **fit_kwargs):
        model = self.hypermodel.build(trial.hyperparameters)
        model.fit(X, y, batch_size=trial.hyperparameters.Choice(
            'batch_size', [16, 32]), **fit_kwargs)

        X_val, y_val = validation_data
        eval_scores = model.evaluate(X_val, y_val)
        return {name: value for name, value in zip(
            model.metrics_names,
            eval_scores)}

In [ ]:
def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.Input(shape = X_train.shape[1:], name = 'inputs'))
    for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
              model.add(tf.keras.layers.Conv2D(hp.Int(
                  'units_{i}'.format(i=i), min_value=32, max_value=128, step=5), (3,3),activation='relu'))
    model.add(tf.keras.layers.Flatten())
    for i in range(hp.Int('n_connections', 1, 3)):
        model.add(tf.keras.layers.Dense(hp.Choice(f'n_nodes',
                                  values=[32,64,128, 256]), activation = 'relu'))
    model.add(tf.keras.layers.Dense(10, activation='softmax', name = 'outputs'))
    model.compile(optimizer = 'adam',loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model

In [ ]:
my_keras_tuner = DeepTuner(
    oracle=kt.oracles.BayesianOptimizationOracle(
        objective=kt.Objective('accuracy', 'max'),
        max_trials=10,
        seed=42),
    hypermodel=build_model,
    overwrite=True,
    project_name='cifar10_tuner')

In [ ]:
my_keras_tuner.search(X_train, y_train, validation_data=(X_val, y_val), epochs=3)

Trial 10 Complete [00h 00m 48s]
accuracy: 0.4708000123500824

Best accuracy So Far: 0.603600025177002
Total elapsed time: 00h 08m 24s


In [ ]:
best_hps = my_keras_tuner.get_best_hyperparameters(num_trials=10)[0]
model = build_model(best_hps)
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_65 (Conv2D)           (None, 30, 30, 102)       2856      
_________________________________________________________________
conv2d_66 (Conv2D)           (None, 28, 28, 42)        38598     
_________________________________________________________________
conv2d_67 (Conv2D)           (None, 26, 26, 67)        25393     
_________________________________________________________________
conv2d_68 (Conv2D)           (None, 24, 24, 37)        22348     
_________________________________________________________________
conv2d_69 (Conv2D)           (None, 22, 22, 52)        17368     
_________________________________________________________________
flatten_11 (Flatten)         (None, 25168)             0         
_________________________________________________________________
dense_16 (Dense)             (None, 256)             

In [ ]:
model.fit(X_train, y_train, batch_size = 32, epochs = 10)

Epoch 1/10
1250/1250 [==============================] - 11s 9ms/step - loss: 1.8227 - accuracy: 0.3901
Epoch 2/10
1250/1250 [==============================] - 11s 9ms/step - loss: 1.1679 - accuracy: 0.5870
Epoch 3/10
1250/1250 [==============================] - 11s 9ms/step - loss: 0.8251 - accuracy: 0.7120
Epoch 4/10
1250/1250 [==============================] - 11s 9ms/step - loss: 0.4566 - accuracy: 0.8432
Epoch 5/10
1250/1250 [==============================] - 11s 9ms/step - loss: 0.2487 - accuracy: 0.9167
Epoch 6/10
1250/1250 [==============================] - 11s 9ms/step - loss: 0.1726 - accuracy: 0.9444
Epoch 7/10
1250/1250 [==============================] - 11s 9ms/step - loss: 0.1430 - accuracy: 0.9531
Epoch 8/10
1250/1250 [==============================] - 11s 9ms/step - loss: 0.1340 - accuracy: 0.9570
Epoch 9/10
1250/1250 [==============================] - 11s 9ms/step - loss: 0.1128 - accuracy: 0.9636
Epoch 10/10
1250/1250 [==============================] - 11s 9ms/step - l

In [ ]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 4ms/step - loss: 2.9473 - accuracy: 0.5482


[2.9473025798797607, 0.5482000112533569]

In [ ]:
save_path = os.getenv('HOME') + '/aiffel/mlops/best_model/1'
fname = os.path.join(save_path, 'model')
model.save(fname)

INFO:tensorflow:Assets written to: /aiffel/aiffel/mlops/best_model/1/model/assets


### 2. Deploy the model

In [ ]:
# data="$(pwd)/aiffel_quest/Exploration_quest/exploration_6/cifar10_saved_model"

In [ ]:
# docker run -t --rm -p 8500:8500 \
#     -v "$data/model:/saved_model" \
#     -e MODEL_NAME=saved_model \
#     tensorflow/serving &

![](https://drive.google.com/uc?export=view&id=1zY12YNbi0XltccS6mxzi4_4wZePcOfe4)

![](https://drive.google.com/uc?export=view&id=1kKEDG8zmTLMOEPILFR0uOnXoI3PYDJ9Z)

배포는 되었으나, 해당 모델을 찾을 수 없다는 에러 발생. 여기서 생각한 솔루션은  
1. 파일 경로 다시 지정,  
2. .py 파일로 변경하고 재업로해서 배포 시도,  
3. Jupyter Notebook Python Stack Extention 설치한 후 위의 과정을 다시 반복,
4. 다른 모델로 시도.

![](https://drive.google.com/uc?export=view&id=1w9LrXueqkk4k1xc0w4tIYpn4SLa7UFvd)

![](https://drive.google.com/uc?export=view&id=1woAdUkPcFHFI3eJ3ClC_BJRIZm0DHeYL)

![](https://drive.google.com/uc?export=view&id=1b0UTcrhin9AWtsrM2wjoZM08CxUikWL-)

### 3. Create a TFLite model

[keras_tuner.ipynb](https://colab.research.google.com/drive/1Uhp4AOLUjvQWVFnsjoloVaSx-_TDJqzi?usp=sharing)

In [ ]:
load_path = os.getenv('HOME') + '/aiffel/mlops/best_model/1/model'
best_model = tf.keras.models.load_model(load_path)

In [ ]:
best_model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_65 (Conv2D)           (None, 30, 30, 102)       2856      
_________________________________________________________________
conv2d_66 (Conv2D)           (None, 28, 28, 42)        38598     
_________________________________________________________________
conv2d_67 (Conv2D)           (None, 26, 26, 67)        25393     
_________________________________________________________________
conv2d_68 (Conv2D)           (None, 24, 24, 37)        22348     
_________________________________________________________________
conv2d_69 (Conv2D)           (None, 22, 22, 52)        17368     
_________________________________________________________________
flatten_11 (Flatten)         (None, 25168)             0         
_________________________________________________________________
dense_16 (Dense)             (None, 256)             

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmphct_l0kf/assets


In [ ]:
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)

signatures = interpreter.get_signature_list()
print(signatures)

{}


In [ ]:
classify_lite = interpreter.get_signature_runner('serving_default')
classify_lite

### 4. Conclusion

### Retrospect

> 개인적으로 Heroku나 Vercel을 활용하는 편이라 전체적인 작업이 조금 번거롭게 느껴졌습니다. Docker와 TFServing을 활용해본 것이 유의미했고, 로컬 환경에서 다양한 방법으로 배포를 시도해본 것이 유익했습니다. 작업 과정에서 ML 모델을 배포함에 있어 TKServing과 Docker를 사용하는 것이 일반적으로 사용하는 방법인지 and/or 엔지니어들이 선호하는 방식인지 궁금했습니다. container의 개념이 흥미로웠고 한편, container나 deploy의 원리가 확립되지 않은 상태에서 단순히 플로우를 경험해보는 것은 무의미하다는 생각을 했습니다. 과정에서 다양한 참고 자료가 필요했고, 참고한 자료는 아래와 같습니다.

### Reference

* [TensorFlow Serving with Docke](https://www.tensorflow.org/tfx/serving/docker)
* [Hosting Models with TF Serving on Docker](https://towardsdatascience.com/hosting-models-with-tf-serving-on-docker-aceff9fbf533)
* [Serving ML Quickly with TensorFlow Serving and Docker](https://medium.com/tensorflow/serving-ml-quickly-with-tensorflow-serving-and-docker-7df7094aa008)
* [How to Serve Machine Learning Models With TensorFlow Serving and Docker](https://neptune.ai/blog/how-to-serve-machine-learning-models-with-tensorflow-serving-and-docker)